# Mistral-7B-Instruct 모델

## "학습” 안 하는 이유
1. 데이터 로딩/전처리:
의료 QA 데이터 등 → 텍스트 조각(Document)으로 분할 및 필터링
2. 임베딩:
Ko-SBERT 등 “사전학습된 임베딩 모델”로 텍스트를 벡터로 변환
(여기서 학습이 아니라 “이미 학습된 모델로 변환”만 하는 것)
3. 벡터DB 생성:
FAISS에 임베딩 벡터 저장
4. 질의응답:
사용자가 질문하면 →
    - (1) 벡터DB에서 유사한 문서 검색
    - (2) Mistral-7B-Instruct 등 대형 언어모델(역시 “사전학습된”)이
검색된 문서 조각들을 참고해서 답변 생성
## 실제 “학습”이란?
- 파인튜닝/미세조정 등은
모델을 직접 추가로 훈련시키는 것 (모델 파라미터가 변함)
- 예시: model.fit(), trainer.train(), “epochs”, “loss” 등이 코드에 나옴

지금 코드의 RAG 구조는:
- 임베딩 모델(KoSBERT): 이미 학습된 걸 “불러와서” 임베딩만 실행
- LLM(Mistral-7B-Instruct 등): 이미 학습된 걸 “불러와서” 답변 생성만 하는 것
- 파인튜닝(학습) 없음

## 요약
“임베딩/벡터DB/질의응답 파이프라인”이지
모델 파인튜닝(학습)은 안 하는 구조

# “RAG 없는 LLM 챗봇”
1. 학습(파인튜닝)이 필요한 경우가 많음
2. LLM(예: Mistral, Llama, GPT 등)만 놓고 사용하면
- → 사전학습 범위 내의 지식만 답변
- → 회사/기관/도메인 고유 정보, 최신 정보 반영 불가
- → 답변 품질, 일관성 낮음

3. **우리 데이터/지식으로 LLM을 ‘맞춤화’**하고 싶으면
→ 파인튜닝(학습)이 필요

# “RAG 있는 LLM 챗봇”
1. 학습(파인튜닝) 없이도 실제 업무·실전 서비스 가능
- 내·외부 문서, 자료를 벡터DB로 embedding해서
- “검색-답변(Retrieval + Generation)” 방식으로 동작
- LLM은 문서에서 답을 찾아서 생성
- 우리 데이터만 잘 embedding하면 바로 적용
(LLM은 사전학습된 상태 그대로 사용)

# 정리
1. RAG 없음 → 파인튜닝 거의 필수
(실무 적용/특화된 답변 원할 때)

2. RAG 있음 → 파인튜닝 필요 없음
(거의 모든 도메인 챗봇, 사내 QA 가능)

# RAG는 “검색+생성” 구조라 우리 데이터를 LLM에 직접 넣어주지 않아도, 최신/도메인 정보로 답변 가능

====================================================================================================================================================================================================

## 임포트 및 환경 설정

In [2]:
import os, json
import pandas as pd
from tqdm import tqdm
from typing import List
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import requests
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline
import re

In [ ]:
# GPU 확인용 코드
# CPU 사용시 느림 현상으로 멈춤 => GPU 사용
import torch

print(torch.cuda.is_available())  # True면 GPU / False면 CPU
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))  # GPU 이름 확인

In [4]:
# protobuf 에러로 인하여 버전 확인용 코드
# import google.protobuf
# print(google..__version__)

# 1. 데이터 불러오기

## 1-1. 네이버 데이터 불러오기

In [5]:
# 네이버 CSV
# naver = pd.read_csv('dataset/naver_health_v1_cleaned.csv')

# 상위 5개 행 확인
# naver.head()

In [6]:
# 네이버 JSON
with open("dataset/naver_docs.json", encoding="utf-8") as f:
    naver = json.load(f)

print("네이버 문서 수:", len(naver))
print("-" * 50)
print("샘플 1개:", naver[0])

네이버 문서 수: 1240
--------------------------------------------------
샘플 1개: {'id': 'naver_0000', 'text': '가랑이통증 [Perineal pain]은(는) 원인에 따라 증상에 다소 차이가 있을 수 있다. *요로감염:  배뇨통  및 빈뇨,  절박뇨 , 골반부위의 둔통 등 *항문주위농양: 발적과 종창, 압통 및 욱신거리는 통증을 동반한 덩이(감염에 의한 경우 열 및 오한을 동반할 수 있음) *음부신경포착증후군: 앉은자세일때 심해지는 통증 *골반바닥근육의 이상: 배뇨, 배변시의 장애 동반 등 증상이 나타나는 질환이며, 관련 진료과는 산부인과입니다.'}


## 1-2. 아산병원 데이터 불러오기

In [7]:
# amc CSV
# naver = pd.read_csv('dataset/amc_disease_v3_cleaned.csv')

# 상위 5개 행 확인
# naver.head()

In [8]:
# AMC JSON
with open("dataset/amc_docs.json", encoding="utf-8") as f:
    amc = json.load(f)

print("AMC 문서 수:", len(amc))
print("-" * 50)
print("샘플 1개:", amc[0])

AMC 문서 수: 1278
--------------------------------------------------
샘플 1개: {'id': 'amc_0000', 'text': '18번 염색체 단완결실 증후군(18p monosomy)은(는) 18번 염색체 단완결실 증후군에는 소두증, 안검 하수, 사시, 양안 격리증, 크고 처진 입, 소하악증, 크고 돌출된 귀, 전전뇌증(10%), 심장 기형, 성장 장애, 충치, 면역글로불린A(IgA) 결핍, 대머리, 터너 증후군(일부 환자) 증상이 동반됩니다. 이 증후군으로 인해 정신 지체가 발생한 경우에는 IQ가 25~75 정도로 나타납니다. 전전뇌증이 없는 경우에는 대부분 성인까지 생존합니다. 증상이 나타나는 질환이며, 관련 진료과는 의학유전학과입니다.'}


## 1-3. medical_knowledge_QA 폴더 데이터 불러오기

In [9]:
# 폴더 경로 리스트
label_dirs = [
    "dataset/medical_knowledge_QA/Training/02.라벨링데이터",
    "dataset/medical_knowledge_QA/Validation/02.라벨링데이터",
]

all_qa_data = []

for label_dir in label_dirs:
    for filename in os.listdir(label_dir):
        if filename.endswith(".json"):
            file_path = os.path.join(label_dir, filename)
            with open(file_path, encoding="utf-8-sig") as f:
                try:
                    data = json.load(f)
                    all_qa_data.append(data)
                except Exception as e:
                    print(f"[오류] {filename}: {e}")

print("총 불러온 문항 수:", len(all_qa_data))
print("-" * 50)
print("예시:", all_qa_data[0])

총 불러온 문항 수: 17280
--------------------------------------------------
예시: {'qa_id': 11, 'domain': 17, 'q_type': 1, 'question': '23세 여자가 3개월 전부터 기침을 한다며 내원했다. 기침은 밤에 누워 자려고 할 때 심해진다고 한다. 1년 전에도 같은 시기에 기침이 3개월 동안 지속되다가 저절로 호전된 병력이 있다. 콧물이나 인후부 불편감은 없으며, 비흡연자이고 복용 중인 약물도 없다. 신체검사에서 혈압 120/80mmHg, 맥박 78회/분, 호흡 18회/분, 체온 36.5°C로 측정되었다. 청진상 호흡음은 정상이었고, 가슴 X선 사진과 코곁굴 X선 사진에서도 이상 소견이 없었다. 폐기능검사 결과는 다음과 같다.  \n- 강제 폐활량(FVC): 정상 예측치의 91%  \n- 1초간 강제날숨유량(FEV1): 정상 예측치의 85%  \n- 1초간 강제날숨유량/강제폐활량(FEV1/FVC): 75%  \n\n이 환자에서 다음으로 시행해야 할 검사는 무엇인가?  \n1) 기관지내시경  \n2) 기관지 확장제 반응 검사  \n3) 가슴 컴퓨터단층촬영(CT) \n4) 메타콜린 기관지유발검사  \n5) 폐 확산능 검사', 'answer': '4) 메타콜린 기관지유발검사'}


## 1-4. medical_legal_corpus 폴더 원천 데이터 불러오기

In [10]:
text_dir = "dataset/medical_legal_corpus/Training/01.원천데이터"
all_texts = []

for filename in os.listdir(text_dir):
    if filename.endswith(".txt"):
        file_path = os.path.join(text_dir, filename)
        with open(file_path, encoding="utf-8-sig") as f:
            text = f.read()
            all_texts.append({"filename": filename, "text": text})

print("총 텍스트 문서 수:", len(all_texts))
print("-" * 50)
print("샘플 문서:", all_texts[0]["filename"])
print("-" * 50)
print(all_texts[0]["text"][:300])

총 텍스트 문서 수: 37507
--------------------------------------------------
샘플 문서: MPA000001.txt
--------------------------------------------------
본 발명에 의한 안마장치는 높이가 조절 가능한 두 개의 받침대 사이에 전후로 이동 가능토록 설치된 안마장치로서, 상기 안마장치는 상기 받침대 내측에 설치된 가이드를 따라 전후로 이동되는 프레임과, 상기 프레임 내부의 일정위치에 고정된 모터와, 상기 모터의 회전축에 설치된 워엄 기어와, 상기 워엄 기어에 맞물려 동력을 전달하게 되는 기어들과, 상기 기어들의 동력을 전달받아 회전되는 회전축과, 상기 회전축에 일정각도 기울어지도록 끼워지고 상부에 사람의 몸체를 안마하게 되는 다수개의 돌기가 방사형으로 형성된 안마판과, 상기 안마판이 전후


## 1-5. 라벨링 QA (Training_medical.json) 불러오기

In [11]:
label_path = (
    "dataset/medical_legal_corpus/Training/02.라벨링데이터/Training_medical.json"
)

with open(label_path, encoding="utf-8-sig") as f:
    legal_qa_data = json.load(f)

qa_items = legal_qa_data.get("data", [])
print("총 QA 항목 수:", len(qa_items))
print("-" * 50)
print("샘플 QA:", qa_items[0])

총 QA 항목 수: 37507
--------------------------------------------------
샘플 QA: {'book_id': 'MPA000001', 'category': '재활의학/물리치료학/작업치료학', 'popularity': 2, 'keyword': ['척추부', '근육', '받침대', '온열치료', '척추'], 'text': '본 발명에 의한 안마장치는 높이가 조절 가능한 두 개의 받침대 사이에 전후로 이동 가능토록 설치된 안마장치로서, 상기 안마장치는 상기 받침대 내측에 설치된 가이드를 따라 전후로 이동되는 프레임과, 상기 프레임 내부의 일정위치에 고정된 모터와, 상기 모터의 회전축에 설치된 워엄 기어와, 상기 워엄 기어에 맞물려 동력을 전달하게 되는 기어들과, 상기 기어들의 동력을 전달받아 회전되는 회전축과, 상기 회전축에 일정각도 기울어지도록 끼워지고 상부에 사람의 몸체를 안마하게 되는 다수개의 돌기가 방사형으로 형성된 안마판과, 상기 안마판이 전후 이동하도록 상기 프레임을 전후방향으로 구동하는 프레임 구동장치를 포함하는 것을 특징으로 한다. 본 발명에 의한 안마장치는 상기 받침대 상부면에 받침대 사이의 공간을 받칠 수 있도록 부드러운 재질로 이루어진 시트가 설치된 것을 특징으로 한다. 본 발명에 의한 안마장치는 상기 안마판에 안마판이 회전되지 않으면서 상하로만 진동하도록 회전축이 끼워지는 부분에 베어링이 설치된 것을 특징으로 한다. 본 발명에 의한 안마장치는 상기 프레임 구동장치가 상기 프레임의 전후에 각각 양단이 연결된 두 개의 체인과, 상기 체인이 걸치게 되는 각각의 스프로킷과, 상기 스프로킷을 구동하는 모터로 구성된 것을 특징으로 한다. 본 발명에 의한 안마장치는 상기 안마판이 사람의 척추의 좌우를 안마할 수 있도록 사람의 척추 양쪽을 따라 두 개가 설치된 것을 특징으로 한다. 본 발명에 의한 안마장치는 상기 안마판이 중앙에 한 개가 설치되되, 그 상면의 돌기가 사람의 척추에 닫지 않도록 형성된 것을 특징으로 

## 1-6. pdf 텍스트 추출 후 문단 단위로 나누기

In [12]:
reader = PdfReader("dataset/antibiotic_guideline_for_longtermcare.pdf")
all_text = ""

for page in reader.pages:
    all_text += page.extract_text() + "\n"

print(all_text[:1000])  # 상위 일부만 출력

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
)
chunks = splitter.split_text(all_text)

print(f"문단 수: {len(chunks)}")
print("-" * 50)
print(chunks[0][:300])  # 첫 문단 확인

감염병 진단 및
항생제 사용지침요양병원
이 지침은 우리나라 요양병원에서 흔히 발생하는 폐렴, 요로감염, 피부연부조직감염 
및 욕창 감염에서 감염병의 적절한 진단과 항생제의 적정 사용을 위해 개발되었으며, 
대한항균요법학회의 자문과 승인을 받았습니다.
 
주요 내용은 요양병원의 현실에서 필요한 검사 및 경험적, 치료적 항생제의 선택에 관한  
것으로, 현장의 활용도를 높이기 위해 기존 지침을 기반으로 단순한 임상경로 형태로 
개발하였습니다.
 
이 지침은 실제 진료 현장에서 개별 환자를 직접 진료하는 의사에게 참고 자료로 제공
하기 위한 것으로, 모든 환자에게 일률적으로 적용하는 것을 권장하지 않습니다.
 
또한, 개인적인 진료 및 교육 목적으로 사용될 수 있지만, 상업적 목적이나 진료 심사, 
임상 의사의 최종적 판단에 대한 적정성 평가 목적으로 사용될 수 없음을 밝힙니다.지침 사용안내 
1 병원획득폐렴  ······································································ 05
 임상경로 및 경험적 항생제 선택
 치료적 항생제의 선택
2  요로감염  ··········································································· 09
 임상경로 및 경험적 항생제 선택
 치료적 항생제의 선택
3  피부연부조직감염  ································································ 13
 임상경로 및 경험적 항생제 선택
 치료적 항생제의 선택
4 욕창감염  ············································································· 27
 임상경로 및 경험적 항생제 선택
 치료적 항생제의 선택
5  신기능에 따른 항생제 용량 ····················································· 21목차
감염병 진단

## 1-7. 약관련 데이터 API(일단 제외)

In [13]:
# import urllib3
# urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [14]:
# import requests
# import json

# # 실제 API URL과 본인 서비스키
# url = "https://apis.data.go.kr/1471000/DrugPrdtPrmsnInfoService06"
# service_key = "j5BnFLQrFMcichD048pxr5OZYe50JKjvUM7DpqQWOGeyKJ1NgIH+4bU6GwY/6kTcPXsqt82F6Tpkhd6BBQpXng=="  # 인코딩 상태로 입력

# params = {
#     "serviceKey": service_key,
#     "pageNo": 1,
#     "numOfRows": 10,
#     "type": "json"  # json 또는 xml (보통 json 추천)
#     # 추가 파라미터는 공공데이터포털 문서 참고
# }

# headers = {
#     "User-Agent": "Mozilla/5.0"
# }

# try:
#     # API 요청
#     response = requests.get(url, params=params, headers=headers, verify=False)
#     response.raise_for_status()
#     data = response.json()

#     # 공공데이터포털은 보통 이렇게 중첩되어 있음
#     # {'response': {'body': {'items': {'item': [...]}}}}
#     items = []
#     if isinstance(data, dict):
#         items = (
#             data.get('body', {}).get('items', {}).get('item')
#             or data.get('response', {}).get('body', {}).get('items', {}).get('item')
#             or data.get('response', {}).get('body', {}).get('items')
#         )
#         # 일부 API는 'item' 없이 리스트일 수도 있음

#         if items:
#             if isinstance(items, list):
#                 for item in items:
#                     # 예시: 생약명, 성분명, 효능 등
#                     print(
#                         "생약명:", item.get('PRDLST_NM'),    # 제품명/생약명
#                         "성분명:", item.get('IFTKN_ATNT_MATR_CN'),  # 주요 성분/주의성분 등
#                         "효능:", item.get('PRIMARY_FNCLTY')  # 효능
#                     )
#             elif isinstance(items, dict):
#                 print(items)
#             else:
#                 print("데이터 형식:", type(items), items)
#         else:
#             print("데이터가 없습니다.")
#     else:
#         print("예상치 못한 데이터 형식입니다.")

# except requests.exceptions.RequestException as e:
#     print(f"API 요청 에러: {e}")
# except json.JSONDecodeError as e:
#     print(f"JSON 디코딩 에러: {e}")
# except Exception as e:
#     print(f"에러 발생: {e}")

# 2. 벡터 DB 생성 및 저장
## 텍스트 추출 및 병합

In [ ]:
# 모든 텍스트를 리스트로 수집
texts = []

# 네이버, 아산병원
texts += [doc["text"] for doc in naver]
texts += [doc["text"] for doc in amc]

# medical_knowledge_QA
for item in all_qa_data:
    if isinstance(item, list):
        for qa in item:
            q = qa.get("question", "")
            a = qa.get("answer", "")
            if q or a:
                texts.append(q + "\n" + a)

# medical_legal_corpus
texts += [doc["text"] for doc in all_texts if isinstance(doc, dict) and "text" in doc]

# 라벨링 QA
texts += [qa.get("text", "") for qa in qa_items if "text" in qa]

print(f"총 수집된 문서 수: {len(texts)}")

총 수집된 문서 수: 77532


## 텍스트 분할

In [ ]:
# 너무 긴 문서는 쪼갬 (임베딩 품질을 위해)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800, chunk_overlap=100, separators=["\n\n", "\n", " ", ""]
)

documents = splitter.create_documents(texts)
print(f"분할된 문서 수: {len(documents)}")

분할된 문서 수: 513583


## 임베딩 및 FAISS 저장

In [ ]:
# Ko-SBERT 임베딩 모델
embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")

# 벡터 DB 생성
db = FAISS.from_documents(documents, embedding=embedding_model)

# 로컬 저장
db.save_local("faiss_medical_knowledge")
print("FAISS 벡터 DB 저장 완료!")

C:\Users\user\AppData\Local\Temp\ipykernel_18080\2390589246.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")
c:\Users\user\Desktop\medical_chatbot\.venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


FAISS 벡터 DB 저장 완료!


# 3. 질의 응답용 Retriever 및 LLM 연결

In [ ]:
# LLM 로딩
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map="auto", torch_dtype="auto"
)

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15,
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)

# 4. 질의 응답 실행

In [ ]:
# 저장된 벡터 DB 로드
db = FAISS.load_local(
    "faiss_medical_knowledge",
    embedding_model,
    allow_dangerous_deserialization=True,  # 직접 만든 벡터DB만 True
)

# Retriever 설정
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# QA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

# 5-1. 사용자 입력에 따른 응답 출력

In [ ]:
# while True:
#     query = input("\n질문을 입력하세요 (종료: exit): ")
#     if query.lower() == "exit":
#         break

#     result = qa_chain({"query": query})
#     print("\n답변:\n", result["result"])
#     print("=" * 200)
#     print("\n참고 문서:")
#     for i, doc in enumerate(result["source_documents"]):
#         print(f"\n[{i+1}] {doc.page_content[:300]}...")

# 5-2. 사용자 입력에 따른 응답 출력

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# 질문 입력창, 버튼, 출력창 생성
question_box = widgets.Text(
    value="",
    placeholder="여기에 질문을 입력하세요",
    description="질문:",
    disabled=False,
)
output_box = widgets.Output()
submit_btn = widgets.Button(description="질문하기")


def on_submit(btn):
    query = question_box.value
    if query.strip().lower() == "exit":
        with output_box:
            clear_output()
            print("종료합니다.")
        return
    with output_box:
        clear_output()
        print("답변을 생성하고 있습니다...")
        result = qa_chain({"query": query})
        print("\n[답변]:", result["result"])
        print("=" * 80)
        print("[참고 문서]")
        for i, doc in enumerate(result["source_documents"]):
            print(f"\n[{i+1}] {doc.page_content[:300]}...")


submit_btn.on_click(on_submit)

# 위젯 표시
display(question_box, submit_btn, output_box)